# 10. Pipeline Internals (Capstone)

**Estimated Time**: ~2 hours

**Prerequisites**: Notebooks 1-9 (this capstone ties together everything you've learned)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the three stages of a Hugging Face pipeline (tokenization, inference, post-processing)
2. **Perform** manual tokenization and understand token IDs, attention masks, and special tokens
3. **Run** model inference manually using PyTorch and interpret raw logits
4. **Apply** softmax to convert logits to probabilities
5. **Build** a custom classification pipeline from scratch with detailed logging

## Setup

Run this cell first. This notebook requires PyTorch and Transformers.

In [ ]:
# Core imports
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F
import numpy as np

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Setup complete!")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

---

# Part 1: Conceptual Foundation

## What Happens Inside a Pipeline?

Throughout this course, you've used `pipeline()` to perform various NLP tasks. But what actually happens when you call it?

```
THE THREE STAGES OF A PIPELINE:

┌─────────────────────────────────────────────────────────────────────────┐
│                                                                         │
│   Input: "I love machine learning!"                                    │
│                                                                         │
│                         ▼                                               │
│   ┌─────────────────────────────────────────────────────────────────┐  │
│   │  STAGE 1: TOKENIZATION                                          │  │
│   │                                                                  │  │
│   │  • Split text into tokens                                       │  │
│   │  • Convert tokens to numerical IDs                              │  │
│   │  • Add special tokens ([CLS], [SEP])                            │  │
│   │  • Create attention mask                                        │  │
│   │                                                                  │  │
│   │  Output: {input_ids: [101, 1045, 2293, ...], attention_mask: ...}│  │
│   └─────────────────────────────────────────────────────────────────┘  │
│                         ▼                                               │
│   ┌─────────────────────────────────────────────────────────────────┐  │
│   │  STAGE 2: MODEL INFERENCE                                       │  │
│   │                                                                  │  │
│   │  • Feed tokenized input to transformer model                    │  │
│   │  • Model processes through all layers                           │  │
│   │  • Output: raw logits (unnormalized scores)                     │  │
│   │                                                                  │  │
│   │  Output: tensor([[-2.45, 3.21]])  # Raw logits                  │  │
│   └─────────────────────────────────────────────────────────────────┘  │
│                         ▼                                               │
│   ┌─────────────────────────────────────────────────────────────────┐  │
│   │  STAGE 3: POST-PROCESSING                                       │  │
│   │                                                                  │  │
│   │  • Apply softmax to get probabilities                           │  │
│   │  • Map indices to labels                                        │  │
│   │  • Format output                                                │  │
│   │                                                                  │  │
│   │  Output: {'label': 'POSITIVE', 'score': 0.96}                   │  │
│   └─────────────────────────────────────────────────────────────────┘  │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

### Why Understand Pipeline Internals?

| Reason | Benefit |
|--------|--------|
| **Debugging** | Understand where errors come from |
| **Optimization** | Batch processing, caching tokenization |
| **Customization** | Modify any stage for specific needs |
| **Understanding** | Know what the model actually "sees" |
| **Research** | Access intermediate representations |

### Connection to Previous Notebooks

This capstone brings together concepts from every previous notebook:

| Notebook | Concept | How It Connects |
|----------|---------|----------------|
| 1 (Fill-Mask) | [MASK] token | Special tokens in tokenization |
| 2 (NER) | Token classification | Per-token logits |
| 3 (QA) | Start/end positions | Position logits |
| 4 (Summarization) | Generation params | Decoding from logits |
| 5 (Text Gen) | Temperature, top-k | Probability manipulation |
| 6 (Zero-Shot) | NLI hypothesis | How classification works |
| 7 (Translation) | Encoder-decoder | Model architecture |
| 8 (Embeddings) | Hidden states | What models produce internally |
| 9 (Sentiment) | Multiple models | Same pipeline, different models |

### Key Terminology

| Term | Definition |
|------|------------|
| **Token** | A piece of text (word, subword, or character) |
| **Token ID** | Numerical representation of a token |
| **Special tokens** | Reserved tokens like [CLS], [SEP], [PAD], [MASK] |
| **Attention mask** | Binary mask indicating real tokens (1) vs padding (0) |
| **Logits** | Raw, unnormalized model outputs |
| **Softmax** | Function that converts logits to probabilities |
| **torch.no_grad()** | Context manager to disable gradient computation |
| **Inference** | Running a model to get predictions (not training) |

### Check Your Understanding

Before moving on, try to answer these questions (answers at the end):

1. What are the three stages of a Hugging Face pipeline?
   - A) Loading, training, saving
   - B) Tokenization, inference, post-processing
   - C) Input, output, evaluation

2. What are logits?
   - A) Probabilities that sum to 1
   - B) Raw, unnormalized model outputs
   - C) Token IDs

3. What does the attention mask do?
   - A) Hides the model's attention patterns
   - B) Indicates which tokens are real vs padding
   - C) Masks random tokens for training

4. Why use torch.no_grad() during inference?
   - A) To make predictions random
   - B) To save memory and speed up computation
   - C) To train the model

---

# Part 2: Basic Implementation

## Loading Model and Tokenizer Separately

Instead of using `pipeline()`, let's load the components individually:

In [ ]:
# Load model and tokenizer separately
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

print(f"Loading model: {model_name}")
print("="*60)

# Load tokenizer (handles Stage 1: Tokenization)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded!")
print(f"  Vocabulary size: {tokenizer.vocab_size:,}")
print(f"  Model max length: {tokenizer.model_max_length}")

# Load model (handles Stage 2: Inference)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print("\nModel loaded!")
print(f"  Number of labels: {model.config.num_labels}")
print(f"  Label mapping: {model.config.id2label}")

## Stage 1: Tokenization Deep Dive

Let's explore what happens during tokenization:

In [ ]:
# Sample text
text = "I love machine learning!"

print(f"Input text: \"{text}\"")
print("="*60)

# Step 1: Tokenize (convert to tokens)
tokens = tokenizer.tokenize(text)
print(f"\n1. Tokenization:")
print(f"   Tokens: {tokens}")
print(f"   Number of tokens: {len(tokens)}")

In [ ]:
# Step 2: Convert tokens to IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"\n2. Token to ID conversion:")
for token, id_ in zip(tokens, token_ids):
    print(f"   '{token}' → {id_}")

In [ ]:
# Step 3: Full tokenization with special tokens
encoded = tokenizer(text, return_tensors="pt")

print(f"\n3. Full encoding (with special tokens):")
print(f"   Keys: {list(encoded.keys())}")
print(f"\n   input_ids: {encoded['input_ids']}")
print(f"   attention_mask: {encoded['attention_mask']}")

# Decode to see the full sequence
decoded = tokenizer.decode(encoded['input_ids'][0])
print(f"\n   Decoded: \"{decoded}\"")

In [ ]:
# Visualize the tokenization process
def visualize_tokenization(text, tokenizer):
    """Show detailed tokenization breakdown."""
    encoded = tokenizer(text, return_tensors="pt")
    input_ids = encoded['input_ids'][0].tolist()
    attention_mask = encoded['attention_mask'][0].tolist()
    
    print(f"Text: \"{text}\"")
    print("="*70)
    print()
    print(f"{'Pos':<5} {'Token ID':<10} {'Token':<20} {'Attention':<10} {'Type'}")
    print("-"*70)
    
    special_ids = {
        tokenizer.cls_token_id: '[CLS] - Classification',
        tokenizer.sep_token_id: '[SEP] - Separator',
        tokenizer.pad_token_id: '[PAD] - Padding',
        tokenizer.mask_token_id: '[MASK] - Mask',
    }
    
    for i, (id_, mask) in enumerate(zip(input_ids, attention_mask)):
        token = tokenizer.decode([id_])
        token_type = special_ids.get(id_, 'Content')
        print(f"{i:<5} {id_:<10} {repr(token):<20} {mask:<10} {token_type}")


visualize_tokenization(text, tokenizer)

### Understanding Special Tokens

```
SPECIAL TOKENS IN BERT/DistilBERT:

┌──────────────────────────────────────────────────────────────────┐
│  [CLS] I love machine learning ! [SEP]                          │
│    │                               │                            │
│    │                               └── End of sequence marker   │
│    │                                                            │
│    └── Start of sequence / Classification token                 │
│        (Used for classification tasks)                          │
│                                                                  │
│  For classification, the model uses the [CLS] token's          │
│  representation to make predictions.                            │
└──────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Show special token IDs
print("Special Token IDs:")
print("="*40)
print(f"[CLS] token: {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"[SEP] token: {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")
print(f"[PAD] token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"[MASK] token: {tokenizer.mask_token} (ID: {tokenizer.mask_token_id})")
print(f"[UNK] token: {tokenizer.unk_token} (ID: {tokenizer.unk_token_id})")

## Stage 2: Model Inference

Now let's pass the tokenized input through the model:

In [ ]:
# Set model to evaluation mode
model.eval()

# Get tokenized input
inputs = tokenizer(text, return_tensors="pt")

print("Model Inference:")
print("="*60)
print(f"Input text: \"{text}\"")
print(f"\nInputs to model:")
print(f"  input_ids shape: {inputs['input_ids'].shape}")
print(f"  attention_mask shape: {inputs['attention_mask'].shape}")

# Run inference (no gradient computation needed)
with torch.no_grad():
    outputs = model(**inputs)

print(f"\nModel output type: {type(outputs)}")
print(f"Output keys: {outputs.keys()}")

In [ ]:
# Examine the raw logits
logits = outputs.logits

print("Raw Logits:")
print("="*60)
print(f"Logits tensor: {logits}")
print(f"Shape: {logits.shape}  # (batch_size, num_labels)")
print()
print("What do these numbers mean?")
print("-"*60)
print("  - Logits are RAW, UNNORMALIZED scores")
print("  - They can be any real number (positive or negative)")
print("  - Higher logit = model is more confident in that class")
print(f"\n  Logit for NEGATIVE (index 0): {logits[0][0].item():.4f}")
print(f"  Logit for POSITIVE (index 1): {logits[0][1].item():.4f}")

## Stage 3: Post-Processing

Convert logits to probabilities and format output:

In [ ]:
# Apply softmax to convert logits to probabilities
probabilities = F.softmax(logits, dim=-1)

print("Softmax Conversion:")
print("="*60)
print(f"\nLogits:        {logits[0].tolist()}")
print(f"Probabilities: {probabilities[0].tolist()}")
print(f"Sum of probs:  {probabilities[0].sum().item():.4f}  (should be 1.0)")

In [ ]:
# Map to labels
predicted_class = torch.argmax(probabilities, dim=-1).item()
predicted_label = model.config.id2label[predicted_class]
confidence = probabilities[0][predicted_class].item()

print("Final Prediction:")
print("="*60)
print(f"\nPredicted class index: {predicted_class}")
print(f"Predicted label: {predicted_label}")
print(f"Confidence: {confidence:.2%}")
print()
print("All class probabilities:")
for i, prob in enumerate(probabilities[0]):
    label = model.config.id2label[i]
    bar = '*' * int(prob.item() * 40)
    print(f"  {label:10s}: {prob.item():.4f} {bar}")

### Comparing Manual vs Pipeline Results

In [ ]:
# Create pipeline for comparison
classifier = pipeline("sentiment-analysis", model=model_name)

# Get pipeline result
pipeline_result = classifier(text)[0]

print("Comparison: Manual vs Pipeline")
print("="*60)
print(f"\nText: \"{text}\"")
print()
print(f"Manual approach:")
print(f"  Label: {predicted_label}")
print(f"  Score: {confidence:.4f}")
print()
print(f"Pipeline approach:")
print(f"  Label: {pipeline_result['label']}")
print(f"  Score: {pipeline_result['score']:.4f}")
print()
print("Results match!" if predicted_label == pipeline_result['label'] else "Results differ!")

---

## Exercise 1: Manual Classification (Guided)

**Difficulty**: Basic | **Time**: 10-15 minutes

**Your task**: Implement a complete manual classification function.

In [ ]:
def manual_classify(text, tokenizer, model, verbose=True):
    """
    Perform classification manually, showing each step.
    
    Args:
        text: Input text to classify
        tokenizer: The tokenizer
        model: The classification model
        verbose: Whether to print detailed output
        
    Returns:
        dict with label and score
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"MANUAL CLASSIFICATION")
        print(f"{'='*60}")
        print(f"Input: \"{text}\"")
    
    # STAGE 1: Tokenization
    if verbose:
        print(f"\n--- Stage 1: Tokenization ---")
    
    inputs = tokenizer(text, return_tensors="pt")
    
    if verbose:
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        print(f"Tokens: {tokens}")
        print(f"Input IDs: {inputs['input_ids'][0].tolist()}")
    
    # STAGE 2: Model Inference
    if verbose:
        print(f"\n--- Stage 2: Model Inference ---")
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    
    logits = outputs.logits
    
    if verbose:
        print(f"Raw logits: {logits[0].tolist()}")
    
    # STAGE 3: Post-processing
    if verbose:
        print(f"\n--- Stage 3: Post-processing ---")
    
    probabilities = F.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    predicted_label = model.config.id2label[predicted_class]
    confidence = probabilities[0][predicted_class].item()
    
    if verbose:
        print(f"Probabilities: {probabilities[0].tolist()}")
        print(f"Predicted: {predicted_label} ({confidence:.2%})")
    
    return {'label': predicted_label, 'score': confidence}


# Test the function
test_texts = [
    "This movie was absolutely fantastic!",
    "Terrible waste of time.",
    "It was okay, nothing special.",
]

for text in test_texts:
    result = manual_classify(text, tokenizer, model)

---

# Part 3: Intermediate Exploration

## Understanding Attention Masks

When processing batches with different lengths, padding is needed:

In [ ]:
# Batch of texts with different lengths
batch_texts = [
    "Short.",
    "This is a medium length sentence.",
    "This is a much longer sentence that has many more words and tokens.",
]

# Tokenize with padding
batch_inputs = tokenizer(
    batch_texts, 
    padding=True,  # Pad to longest in batch
    return_tensors="pt"
)

print("Batch Tokenization with Padding:")
print("="*70)
print(f"\nInput shapes:")
print(f"  input_ids: {batch_inputs['input_ids'].shape}")
print(f"  attention_mask: {batch_inputs['attention_mask'].shape}")

In [ ]:
# Visualize padding and attention masks
def visualize_batch_padding(texts, tokenizer):
    """Visualize how padding and attention masks work."""
    inputs = tokenizer(texts, padding=True, return_tensors="pt")
    
    print("Batch Padding Visualization:")
    print("="*80)
    
    for i, text in enumerate(texts):
        ids = inputs['input_ids'][i].tolist()
        mask = inputs['attention_mask'][i].tolist()
        
        print(f"\nText {i+1}: \"{text}\"")
        print("-"*80)
        
        # Show tokens with attention mask
        visual = []
        for id_, m in zip(ids, mask):
            token = tokenizer.decode([id_])
            if m == 1:
                visual.append(f"[{token}]")
            else:
                visual.append(f"({token})")
        
        print(f"  Tokens:    {' '.join(visual)}")
        print(f"  Mask:      {mask}")
        print(f"  Real tokens: {sum(mask)}, Padding: {len(mask) - sum(mask)}")
    
    print("\nLegend: [token] = real, (token) = padding")


visualize_batch_padding(batch_texts, tokenizer)

### Why Attention Masks Matter

```
WITHOUT ATTENTION MASK:
┌────────────────────────────────────────────────────────────────┐
│  Model would "see" padding tokens as real content!            │
│                                                                │
│  [CLS] Short . [SEP] [PAD] [PAD] [PAD] [PAD]                  │
│     ↓                   ↓     ↓     ↓     ↓                   │
│  Model attends to everything, including meaningless padding   │
└────────────────────────────────────────────────────────────────┘

WITH ATTENTION MASK:
┌────────────────────────────────────────────────────────────────┐
│  Model only "sees" real tokens                                │
│                                                                │
│  [CLS] Short . [SEP] [PAD] [PAD] [PAD] [PAD]                  │
│     1    1   1   1     0     0     0     0   ← Attention mask │
│     ↓    ↓   ↓   ↓     ✗     ✗     ✗     ✗                    │
│  Model ignores padding tokens completely                      │
└────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Process the batch
model.eval()
with torch.no_grad():
    outputs = model(**batch_inputs)

# Get predictions for all items in batch
probabilities = F.softmax(outputs.logits, dim=-1)

print("\nBatch Predictions:")
print("="*70)

for i, text in enumerate(batch_texts):
    predicted_class = torch.argmax(probabilities[i]).item()
    label = model.config.id2label[predicted_class]
    score = probabilities[i][predicted_class].item()
    
    print(f"\n\"{text}\"")
    print(f"  → {label} ({score:.2%})")

## Exploring Tokenizer Behavior

Different inputs are tokenized differently:

In [ ]:
# Test various tokenization scenarios
test_cases = [
    "Hello world",  # Simple
    "Hello World",  # Capitalization
    "unbelievable",  # Subword tokenization
    "antidisestablishmentarianism",  # Long word
    "COVID-19",  # Special characters
    "I'm don't won't",  # Contractions
    "🔥 This is fire! 🔥",  # Emojis
    "supercalifragilisticexpialidocious",  # Very long word
]

print("Tokenization Scenarios:")
print("="*80)

for text in test_cases:
    tokens = tokenizer.tokenize(text)
    print(f"\n\"{text}\"")
    print(f"  → {tokens}")
    print(f"  → {len(tokens)} tokens")

### Understanding Subword Tokenization

```
WHY SUBWORD TOKENIZATION?

Problem: Vocabulary can't contain every possible word
Solution: Break unknown words into known subwords

Example: "unbelievable" → ["un", "##believ", "##able"]

The ## prefix means "continuation of previous token"

Benefits:
• Handle any word, even misspellings
• Share knowledge between related words
• Manageable vocabulary size
```

In [ ]:
# Show subword tokenization in detail
def analyze_subword_tokenization(word, tokenizer):
    """Analyze how a word is broken into subwords."""
    tokens = tokenizer.tokenize(word)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    print(f"\nWord: \"{word}\"")
    print("-"*40)
    print(f"{'Token':<20} {'ID':<10} {'Type'}")
    print("-"*40)
    
    for token, id_ in zip(tokens, token_ids):
        if token.startswith('##'):
            token_type = 'Continuation'
        else:
            token_type = 'Start of word'
        print(f"{token:<20} {id_:<10} {token_type}")


# Analyze some words
words_to_analyze = [
    "unbelievable",
    "preprocessing",
    "transformers",
    "tokenization",
]

for word in words_to_analyze:
    analyze_subword_tokenization(word, tokenizer)

---

## Exercise 2: Batch Processing (Semi-guided)

**Difficulty**: Intermediate | **Time**: 15-20 minutes

**Your task**: Implement efficient batch processing with proper padding and attention masks.

In [ ]:
def batch_classify(texts, tokenizer, model, batch_size=4):
    """
    Classify multiple texts efficiently using batching.
    
    Args:
        texts: List of texts to classify
        tokenizer: The tokenizer
        model: The classification model
        batch_size: Number of texts per batch
        
    Returns:
        List of dicts with label and score for each text
    """
    results = []
    model.eval()
    
    # Process in batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        
        # Tokenize batch with padding
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        
        # Run inference
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Post-process
        probabilities = F.softmax(outputs.logits, dim=-1)
        
        # Get predictions for each item in batch
        for j in range(len(batch_texts)):
            pred_class = torch.argmax(probabilities[j]).item()
            results.append({
                'text': batch_texts[j],
                'label': model.config.id2label[pred_class],
                'score': probabilities[j][pred_class].item(),
                'all_scores': {model.config.id2label[k]: probabilities[j][k].item() 
                              for k in range(len(probabilities[j]))}
            })
    
    return results


# Test with multiple texts
test_texts = [
    "I absolutely loved this movie!",
    "Terrible experience, never again.",
    "It was okay, nothing special.",
    "The best purchase I've ever made!",
    "Complete waste of money.",
    "Pretty good, would recommend.",
    "Not what I expected.",
    "Exceeded all my expectations!",
]

results = batch_classify(test_texts, tokenizer, model, batch_size=3)

print("Batch Classification Results:")
print("="*70)

for r in results:
    print(f"\n\"{r['text']}\"")
    print(f"  → {r['label']} ({r['score']:.2%})")

---

# Part 4: Advanced Topics

## Accessing Hidden States

Models produce more than just logits - we can access the internal representations:

In [ ]:
# Load model with output_hidden_states enabled
model_with_hidden = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    output_hidden_states=True
)

text = "I love machine learning!"
inputs = tokenizer(text, return_tensors="pt")

model_with_hidden.eval()
with torch.no_grad():
    outputs = model_with_hidden(**inputs)

print("Model Outputs with Hidden States:")
print("="*60)
print(f"\nOutput keys: {outputs.keys()}")
print(f"\nLogits shape: {outputs.logits.shape}")
print(f"Number of hidden state layers: {len(outputs.hidden_states)}")
print(f"Each hidden state shape: {outputs.hidden_states[0].shape}")

In [ ]:
# Analyze hidden states
print("Hidden State Analysis:")
print("="*60)

# Get the final layer's hidden states
final_hidden = outputs.hidden_states[-1]
print(f"\nFinal hidden state shape: {final_hidden.shape}")
print(f"  → (batch_size, sequence_length, hidden_size)")

# The [CLS] token representation (used for classification)
cls_representation = final_hidden[0, 0, :]
print(f"\n[CLS] token representation shape: {cls_representation.shape}")
print(f"First 10 values: {cls_representation[:10].tolist()}")

### Understanding Layer-wise Representations

```
LAYER PROGRESSION:

Layer 0 (Embedding): Raw token embeddings
       ↓
Layer 1: Basic patterns (syntax)
       ↓
Layer 2-3: More complex patterns
       ↓
Layer 4-5: Semantic understanding
       ↓
Final Layer: Task-specific representations
       ↓
Classification Head: Logits

Each layer builds on the previous one,
creating increasingly abstract representations.
```

## Custom Post-Processing

Customize how outputs are processed:

In [ ]:
def custom_postprocess(logits, config, threshold=0.7, temperature=1.0):
    """
    Custom post-processing with configurable options.
    
    Args:
        logits: Raw model outputs
        config: Model config with label mapping
        threshold: Confidence threshold for predictions
        temperature: Temperature for softmax (higher = more uniform)
        
    Returns:
        dict with prediction details
    """
    # Apply temperature scaling
    scaled_logits = logits / temperature
    
    # Convert to probabilities
    probabilities = F.softmax(scaled_logits, dim=-1)
    
    # Get prediction
    max_prob, pred_class = torch.max(probabilities, dim=-1)
    max_prob = max_prob.item()
    pred_class = pred_class.item()
    
    # Apply threshold
    if max_prob < threshold:
        label = "UNCERTAIN"
        confident = False
    else:
        label = config.id2label[pred_class]
        confident = True
    
    return {
        'label': label,
        'score': max_prob,
        'confident': confident,
        'temperature': temperature,
        'all_probs': {config.id2label[i]: probabilities[0][i].item() 
                      for i in range(len(config.id2label))}
    }


# Test with different temperatures
text = "It was okay, I guess."
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

print(f"Text: \"{text}\"")
print("="*60)

for temp in [0.5, 1.0, 2.0]:
    result = custom_postprocess(outputs.logits, model.config, temperature=temp)
    print(f"\nTemperature: {temp}")
    print(f"  Label: {result['label']}")
    print(f"  Score: {result['score']:.4f}")
    print(f"  Confident: {result['confident']}")
    print(f"  All probs: {result['all_probs']}")

### Temperature Effect on Probabilities

```
TEMPERATURE SCALING:

Low temperature (< 1.0):
  - Makes distribution sharper
  - Increases confidence in top prediction
  - More "decisive"

Temperature = 1.0:
  - Standard softmax
  - No scaling

High temperature (> 1.0):
  - Makes distribution flatter
  - Decreases confidence
  - More "uncertain"

Example:
Logits: [2.0, 1.0]

T=0.5: [0.88, 0.12]  ← Very confident
T=1.0: [0.73, 0.27]  ← Normal
T=2.0: [0.62, 0.38]  ← Less confident
```

---

## Exercise 3: Build a Logging Pipeline (Independent)

**Difficulty**: Advanced | **Time**: 15-20 minutes

**Your task**: Build a pipeline wrapper that logs every step of the process.

In [ ]:
# YOUR CODE HERE

import time

class LoggingPipeline:
    """
    A pipeline wrapper that logs every step.
    """
    
    def __init__(self, tokenizer, model, log_level='detailed'):
        """
        Initialize the logging pipeline.
        
        Args:
            tokenizer: The tokenizer
            model: The classification model
            log_level: 'minimal', 'standard', or 'detailed'
        """
        self.tokenizer = tokenizer
        self.model = model
        self.log_level = log_level
        self.logs = []
    
    def _log(self, stage, message, data=None):
        """Add a log entry."""
        entry = {
            'timestamp': time.time(),
            'stage': stage,
            'message': message,
            'data': data
        }
        self.logs.append(entry)
    
    def classify(self, text):
        """
        Classify text with logging.
        
        Returns:
            dict with prediction and logs
        """
        self.logs = []  # Reset logs
        start_time = time.time()
        
        # STAGE 1: Tokenization
        self._log('tokenization', 'Starting tokenization', {'text': text})
        
        tok_start = time.time()
        inputs = self.tokenizer(text, return_tensors="pt")
        tok_time = time.time() - tok_start
        
        tokens = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        self._log('tokenization', f'Tokenized into {len(tokens)} tokens', {
            'tokens': tokens,
            'input_ids': inputs['input_ids'][0].tolist(),
            'time_ms': tok_time * 1000
        })
        
        # STAGE 2: Inference
        self._log('inference', 'Starting model inference')
        
        inf_start = time.time()
        self.model.eval()
        with torch.no_grad():
            outputs = self.model(**inputs)
        inf_time = time.time() - inf_start
        
        logits = outputs.logits
        self._log('inference', 'Model inference complete', {
            'logits': logits[0].tolist(),
            'time_ms': inf_time * 1000
        })
        
        # STAGE 3: Post-processing
        self._log('postprocess', 'Starting post-processing')
        
        post_start = time.time()
        probabilities = F.softmax(logits, dim=-1)
        pred_class = torch.argmax(probabilities, dim=-1).item()
        label = self.model.config.id2label[pred_class]
        score = probabilities[0][pred_class].item()
        post_time = time.time() - post_start
        
        self._log('postprocess', 'Post-processing complete', {
            'probabilities': probabilities[0].tolist(),
            'prediction': {'label': label, 'score': score},
            'time_ms': post_time * 1000
        })
        
        total_time = time.time() - start_time
        self._log('complete', 'Classification complete', {
            'total_time_ms': total_time * 1000
        })
        
        return {
            'label': label,
            'score': score,
            'logs': self.logs
        }
    
    def print_logs(self, result=None):
        """Print formatted logs."""
        logs = result['logs'] if result else self.logs
        
        print("\n" + "="*70)
        print(" PIPELINE EXECUTION LOG")
        print("="*70)
        
        for log in logs:
            stage = log['stage'].upper()
            msg = log['message']
            
            print(f"\n[{stage}] {msg}")
            
            if self.log_level == 'detailed' and log['data']:
                for key, value in log['data'].items():
                    if key == 'tokens' and len(value) > 10:
                        print(f"  {key}: {value[:5]} ... {value[-5:]}")
                    elif isinstance(value, list) and len(value) > 10:
                        print(f"  {key}: [{value[0]}, ..., {value[-1]}] (len={len(value)})")
                    elif key.endswith('_ms'):
                        print(f"  {key}: {value:.2f}ms")
                    else:
                        print(f"  {key}: {value}")
        
        print("\n" + "="*70)


# Test the logging pipeline
logging_pipe = LoggingPipeline(tokenizer, model, log_level='detailed')

result = logging_pipe.classify("This product is absolutely amazing!")
logging_pipe.print_logs(result)

print(f"\nFinal result: {result['label']} ({result['score']:.2%})")

---

# Part 5: Mini-Project

## Project: Custom Pipeline Builder

**Scenario**: You're building a production classification system that needs:
- Detailed logging for debugging
- Configurable confidence thresholds
- Batch processing support
- Performance metrics

**Your goal**: Build a complete `CustomClassificationPipeline` class that implements all three stages from scratch with full configurability.

In [ ]:
# MINI-PROJECT: Custom Pipeline Builder
# =====================================

import time
from dataclasses import dataclass
from typing import List, Dict, Optional, Any

@dataclass
class PipelineConfig:
    """Configuration for the custom pipeline."""
    confidence_threshold: float = 0.5
    temperature: float = 1.0
    max_length: int = 512
    batch_size: int = 8
    return_all_scores: bool = False
    log_level: str = 'standard'  # 'minimal', 'standard', 'detailed'


class CustomClassificationPipeline:
    """
    A fully customizable text classification pipeline.
    
    Features:
    - Configurable confidence thresholds
    - Temperature scaling
    - Batch processing
    - Detailed logging and metrics
    """
    
    def __init__(self, model_name: str, config: Optional[PipelineConfig] = None):
        """
        Initialize the pipeline.
        
        Args:
            model_name: HuggingFace model name or path
            config: Pipeline configuration
        """
        self.config = config or PipelineConfig()
        self.model_name = model_name
        
        # Load model and tokenizer
        self._log("Loading tokenizer...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        self._log("Loading model...")
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.model.eval()
        
        # Store label mapping
        self.id2label = self.model.config.id2label
        self.label2id = self.model.config.label2id
        
        # Metrics tracking
        self.metrics = {
            'total_predictions': 0,
            'total_time_ms': 0,
            'avg_time_ms': 0,
            'confident_predictions': 0,
            'uncertain_predictions': 0,
        }
        
        self._log(f"Pipeline initialized with {len(self.id2label)} labels: {list(self.id2label.values())}")
    
    def _log(self, message: str, level: str = 'standard'):
        """Log a message based on log level."""
        levels = {'minimal': 0, 'standard': 1, 'detailed': 2}
        if levels.get(level, 1) <= levels.get(self.config.log_level, 1):
            print(f"[Pipeline] {message}")
    
    def tokenize(self, texts: List[str]) -> Dict[str, torch.Tensor]:
        """
        Stage 1: Tokenization.
        
        Args:
            texts: List of input texts
            
        Returns:
            Dict with input_ids and attention_mask tensors
        """
        self._log(f"Tokenizing {len(texts)} text(s)...", 'detailed')
        
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.config.max_length,
            return_tensors="pt"
        )
        
        self._log(f"Tokenized shape: {inputs['input_ids'].shape}", 'detailed')
        return inputs
    
    def infer(self, inputs: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Stage 2: Model inference.
        
        Args:
            inputs: Tokenized inputs
            
        Returns:
            Logits tensor
        """
        self._log("Running model inference...", 'detailed')
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        self._log(f"Logits shape: {outputs.logits.shape}", 'detailed')
        return outputs.logits
    
    def postprocess(self, logits: torch.Tensor) -> List[Dict[str, Any]]:
        """
        Stage 3: Post-processing.
        
        Args:
            logits: Raw model outputs
            
        Returns:
            List of prediction dictionaries
        """
        self._log("Post-processing outputs...", 'detailed')
        
        # Apply temperature scaling
        scaled_logits = logits / self.config.temperature
        
        # Convert to probabilities
        probabilities = F.softmax(scaled_logits, dim=-1)
        
        results = []
        for i in range(len(probabilities)):
            probs = probabilities[i]
            max_prob, pred_class = torch.max(probs, dim=-1)
            max_prob = max_prob.item()
            pred_class = pred_class.item()
            
            # Apply threshold
            confident = max_prob >= self.config.confidence_threshold
            label = self.id2label[pred_class] if confident else "UNCERTAIN"
            
            result = {
                'label': label,
                'score': max_prob,
                'confident': confident,
                'predicted_class': pred_class,
            }
            
            if self.config.return_all_scores:
                result['all_scores'] = {
                    self.id2label[j]: probs[j].item()
                    for j in range(len(probs))
                }
            
            results.append(result)
            
            # Update metrics
            if confident:
                self.metrics['confident_predictions'] += 1
            else:
                self.metrics['uncertain_predictions'] += 1
        
        return results
    
    def __call__(self, texts, **kwargs) -> List[Dict[str, Any]]:
        """
        Run the full pipeline.
        
        Args:
            texts: Single text or list of texts
            **kwargs: Override config options
            
        Returns:
            List of prediction dictionaries
        """
        # Handle single text input
        single_input = isinstance(texts, str)
        if single_input:
            texts = [texts]
        
        # Override config if needed
        orig_config = {}
        for key, value in kwargs.items():
            if hasattr(self.config, key):
                orig_config[key] = getattr(self.config, key)
                setattr(self.config, key, value)
        
        start_time = time.time()
        
        all_results = []
        
        # Process in batches
        for i in range(0, len(texts), self.config.batch_size):
            batch = texts[i:i + self.config.batch_size]
            self._log(f"Processing batch {i//self.config.batch_size + 1} ({len(batch)} items)", 'detailed')
            
            # Run pipeline stages
            inputs = self.tokenize(batch)
            logits = self.infer(inputs)
            results = self.postprocess(logits)
            
            # Add original text to results
            for j, result in enumerate(results):
                result['text'] = batch[j]
            
            all_results.extend(results)
        
        # Update metrics
        elapsed = (time.time() - start_time) * 1000
        self.metrics['total_predictions'] += len(texts)
        self.metrics['total_time_ms'] += elapsed
        self.metrics['avg_time_ms'] = (
            self.metrics['total_time_ms'] / self.metrics['total_predictions']
        )
        
        self._log(f"Processed {len(texts)} text(s) in {elapsed:.2f}ms")
        
        # Restore config
        for key, value in orig_config.items():
            setattr(self.config, key, value)
        
        return all_results[0] if single_input else all_results
    
    def get_metrics(self) -> Dict[str, Any]:
        """Get pipeline metrics."""
        return self.metrics.copy()
    
    def reset_metrics(self):
        """Reset metrics."""
        self.metrics = {
            'total_predictions': 0,
            'total_time_ms': 0,
            'avg_time_ms': 0,
            'confident_predictions': 0,
            'uncertain_predictions': 0,
        }


# Create custom pipeline
config = PipelineConfig(
    confidence_threshold=0.7,
    temperature=1.0,
    return_all_scores=True,
    log_level='standard'
)

custom_pipe = CustomClassificationPipeline(
    "distilbert-base-uncased-finetuned-sst-2-english",
    config=config
)

In [ ]:
# Test single prediction
result = custom_pipe("I absolutely love this product!")

print("\nSingle Prediction Result:")
print("="*50)
print(f"Text: \"{result['text']}\"")
print(f"Label: {result['label']}")
print(f"Score: {result['score']:.4f}")
print(f"Confident: {result['confident']}")
if 'all_scores' in result:
    print(f"All scores: {result['all_scores']}")

In [ ]:
# Test batch prediction
test_texts = [
    "This is the best purchase I've ever made!",
    "Terrible quality, completely disappointed.",
    "It's okay, nothing special.",
    "Exceeded all my expectations!",
    "Would not recommend to anyone.",
    "Pretty good for the price.",
    "Meh.",
    "Absolutely fantastic experience!",
]

results = custom_pipe(test_texts)

print("\nBatch Prediction Results:")
print("="*70)

for r in results:
    conf_marker = "✓" if r['confident'] else "?"
    print(f"\n[{conf_marker}] \"{r['text']}\"")
    print(f"    → {r['label']} ({r['score']:.2%})")

In [ ]:
# View metrics
metrics = custom_pipe.get_metrics()

print("\nPipeline Metrics:")
print("="*50)
for key, value in metrics.items():
    if 'time' in key:
        print(f"{key}: {value:.2f}ms")
    else:
        print(f"{key}: {value}")

In [ ]:
# Test with different configurations
print("\nTesting Different Configurations:")
print("="*70)

test_text = "It was okay, I guess."

# High threshold
result_high = custom_pipe(test_text, confidence_threshold=0.9)
print(f"\nHigh threshold (0.9):")
print(f"  {result_high['label']} ({result_high['score']:.4f})")

# Low threshold
result_low = custom_pipe(test_text, confidence_threshold=0.3)
print(f"\nLow threshold (0.3):")
print(f"  {result_low['label']} ({result_low['score']:.4f})")

# Different temperatures
for temp in [0.5, 1.0, 2.0]:
    result_temp = custom_pipe(test_text, temperature=temp, confidence_threshold=0.5)
    print(f"\nTemperature {temp}:")
    print(f"  {result_temp['label']} ({result_temp['score']:.4f})")

### Extension Ideas

If you want to extend this project further:

1. **GPU Support**: Move tensors to GPU for faster inference
2. **Caching**: Cache tokenization for repeated texts
3. **Async Processing**: Implement async batch processing
4. **Model Ensemble**: Support multiple models with voting
5. **Export**: Save predictions to CSV/JSON
6. **Visualization**: Plot confidence distributions

---

# Part 6: Wrap-Up

## Key Takeaways

1. **Pipelines have three stages**:
   - Tokenization: Text → Token IDs + Attention Mask
   - Inference: Token IDs → Raw Logits
   - Post-processing: Logits → Probabilities → Labels

2. **Tokenization involves**:
   - Splitting text into tokens (subword tokenization)
   - Adding special tokens ([CLS], [SEP])
   - Creating attention masks for padding

3. **Logits are raw, unnormalized outputs** - use softmax to convert to probabilities

4. **torch.no_grad()** is essential during inference to save memory and speed up computation

5. **Understanding internals enables**:
   - Better debugging
   - Custom post-processing
   - Batch processing optimization
   - Access to intermediate representations

## How This Connects to Previous Notebooks

| Notebook | Key Pipeline Aspect |
|----------|--------------------|
| 1 (Fill-Mask) | [MASK] token handling |
| 2 (NER) | Per-token predictions |
| 3 (QA) | Position-based logits |
| 4-5 (Generation) | Autoregressive decoding |
| 6-9 (Classification) | Sequence classification |
| **10 (This)** | **All of the above!** |

## Common Mistakes to Avoid

| Mistake | Why It's a Problem |
|---------|-------------------|
| Forgetting attention_mask | Model attends to padding tokens |
| Not using torch.no_grad() | Wastes memory, slower |
| Confusing logits and probs | Logits aren't normalized |
| Ignoring batch dimension | Shape mismatch errors |
| Not setting model.eval() | Dropout still active |

---

## Solutions

### Check Your Understanding (Quiz Answers)

1. **B) Tokenization, inference, post-processing** - The three stages of a Hugging Face pipeline
2. **B) Raw, unnormalized model outputs** - Logits need softmax to become probabilities
3. **B) Indicates which tokens are real vs padding** - Tells the model which tokens to attend to
4. **B) To save memory and speed up computation** - No need to track gradients during inference

In [ ]:
# Summary of key functions from this notebook:

summary = {
    'Tokenization': {
        'tokenizer.tokenize(text)': 'Split text into tokens',
        'tokenizer.convert_tokens_to_ids()': 'Convert tokens to IDs',
        'tokenizer(text, return_tensors="pt")': 'Full encoding with special tokens',
        'tokenizer.decode(ids)': 'Convert IDs back to text',
    },
    'Inference': {
        'model.eval()': 'Set model to evaluation mode',
        'torch.no_grad()': 'Disable gradient computation',
        'model(**inputs)': 'Run forward pass',
    },
    'Post-processing': {
        'F.softmax(logits, dim=-1)': 'Convert logits to probabilities',
        'torch.argmax()': 'Get predicted class',
        'model.config.id2label': 'Map class index to label',
    },
}

print("Key Functions Summary:")
print("="*60)

for stage, functions in summary.items():
    print(f"\n{stage}:")
    for func, desc in functions.items():
        print(f"  {func}")
        print(f"    → {desc}")

---

## Congratulations!

You've completed the entire 10-notebook course on Hugging Face Transformers! You now have:

- **Foundational knowledge** of NLP tasks and transformer architectures
- **Practical experience** with pipelines, models, and tokenizers
- **Deep understanding** of how everything works under the hood

## Additional Resources

- [Hugging Face Transformers Documentation](https://huggingface.co/docs/transformers)
- [Hugging Face Course](https://huggingface.co/course)
- [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)
- [Tokenizers Library](https://huggingface.co/docs/tokenizers)
- [Model Hub](https://huggingface.co/models)